#### Import libraries

In [1]:
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD
from keras.models import Model

Using TensorFlow backend.


In [2]:
K.set_learning_phase(0)

#### Prebuild model with pre-trained weights on imagenet

In [3]:
model = VGG16(weights='imagenet', include_top=True)

In [4]:
config = model.get_config()
weights = model.get_weights()
new_model = Model.from_config(config)
new_model.set_weights(weights)

#### Exporting the model 

In [5]:
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter

In [6]:
export_path = "./simple_model/1"
builder = saved_model_builder.SavedModelBuilder(export_path)

prediction_signature = predict_signature_def(inputs={"import/input_2:0": new_model.input},
                                  outputs={'scores': new_model.output})

with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={'predict': prediction_signature})
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./simple_model/1/saved_model.pb
